In [1]:
import csv
def input_database(file_name):
    
    with open(file_name) as csv_file:
        read_csv = csv.reader(csv_file, delimiter=',')
        next(read_csv)
        rows = []
        for row in read_csv:
            rows.append(row[1])
            
    return rows

In [2]:
def all_distinct_items(rows):
    items_in_row = []
    for row in rows:
        items_in_row.extend(row.split(","))
        
    return list(set(items_in_row))

In [3]:
def string_to_int(all_rows, distinct_items):
    rows = []
    for row in all_rows:
        rows.append(sorted([distinct_items.index(item) for item in row.split(",")]))
    return rows

In [4]:
def find_support_from_items(data, items):
    support_count = 0
    for row in data:
        if set(items).issubset(row):
            support_count += 1
    return support_count

In [5]:
def all_possible_subset(s):
    len_s = len(s)
    
    all_set = []
    
    for i in range(1 << len_s):
        
        all_set.append([s[j] for j in range(len_s) if (i & (1 << j))])
        
    return all_set

In [6]:
def find_association_from_items(data, freq_items, distinct_items, min_confidence):
    association_list = []
    for item in freq_items:
        all_set = all_possible_subset(item)
        for p in all_set:
            left_hand_side = set(p)
            right_hand_side = set(item) - left_hand_side
            if len(left_hand_side) and len(right_hand_side):
                support_lhs = find_support_from_items(data, left_hand_side)
                if support_lhs == 0:
                    continue
                confidence = find_support_from_items(data, item) * 100 / support_lhs
                if confidence < min_confidence:
                    continue
                lhs_items = [distinct_items[i] for i in left_hand_side]
                rhs_items = [distinct_items[i] for i in right_hand_side]
                association_list.append((lhs_items, rhs_items, confidence))
    return association_list

In [7]:
def comparing_support_value(data, elements, min_support):
    frequent_items = []
    non_frequent_items = []
    if elements is not None:
        for e in elements:
            if(find_support_from_items(data, e) >= min_support):
                frequent_items.append(e)
            else:
                non_frequent_items.append(e)
    return frequent_items, non_frequent_items

In [8]:
def check_non_frequent_item(element, non_freq_elements):
    all_pos = all_possible_subset(element)
    for item in all_pos:
        if item in non_freq_elements:
            return True
    return False

In [9]:
def next_level_items(pos_freq_elements, neg_freq_elements):
    next_level_items = {}
    total_pos_items = len(pos_freq_elements)
    if total_pos_items == 0:
        return []
    len_each_item = len(pos_freq_elements[0])
    for left in range(0, total_pos_items):
        for right in range(left+1, total_pos_items):
            merged = tuple(sorted(set(pos_freq_elements[left]).union(set(pos_freq_elements[right]))))
            if len(merged) == len_each_item + 1 and not check_non_frequent_item(merged, neg_freq_elements):
                next_level_items[merged] = 1
    return [list(i) for i in next_level_items.keys()]

In [10]:
def apriori_algorithm(data, item_size, min_support, min_confidence):
    freq_items = []
    non_freq_items = []
    items = range(0, item_size)
    min_support_value = int(len(data) * min_support / 100.0)
    new_freq_elements = [[i] for i in items]
    pos_freq_elements, neg_freq_elements = comparing_support_value(data, new_freq_elements, min_support_value)
    freq_items.extend(pos_freq_elements)
    non_freq_items.extend(neg_freq_elements)
        
    for k in range(2,item_size+1):
        new_freq_elements = next_level_items(pos_freq_elements, non_freq_items)
        pos_freq_elements, neg_freq_elements = comparing_support_value(data, new_freq_elements, min_support_value)
        if not pos_freq_elements:
            break
        freq_items.extend(pos_freq_elements)
        non_freq_items.extend(neg_freq_elements)
    
    return freq_items

In [11]:
def orientation(items, n): 
    if n == 0: 
        return [[]] 
      
    lst = [] 
    for i in range(0, len(items)): 

        m = items[i] 
        rest = items[i + 1:] 
          
        for p in orientation(rest, n-1): 
            lst.append([m]+p) 
              
    return lst

In [12]:
def brute_force_method(data, item_size, min_support, min_confidence):
    freq_items = []
    items = range(0, item_size)
    min_support_value = int(len(data) * min_support / 100.0)
    
    for k in range(1,item_size+1):
        new_freq_elements = list(orientation(items,k))
        print(len(new_freq_elements))
        new_freq_items = list(filter(lambda x: find_support_from_items(data, x) >= min_support_value, new_freq_elements))
        if not new_freq_items:
            break
        freq_items.extend(new_freq_items)
    return freq_items

In [13]:
def user_input():
    min_support = input("Please provide minimum Support value (Only Value): ")
    min_confidence = input("Please provide minimum Confidence value (Only Value): ")
    return min_support, min_confidence

In [14]:
def print_association_rule(file_name, min_support, min_confidence):
    all_rows = input_database(file_name) # load_database()
    distinct_items = all_distinct_items(all_rows)
    num_distinct_items = len(distinct_items)
    data = string_to_int(all_rows, distinct_items)
    
    freq_items = apriori_algorithm(data, num_distinct_items, min_support, min_confidence)
    association_list = find_association_from_items(data, freq_items, distinct_items, min_confidence)
    print("Total number of items in association list is {}".format(len(association_list)))
    for item in association_list:
        lhs_items, rhs_items, confidence = item
        print("{} -->> {} : {}".format(lhs_items, rhs_items, confidence))

In [16]:
for index in range(1,6):
    print("For Database Number # {} ".format(index))
    file_name = 'database_{}.csv'.format(index)
    min_support, min_confidence = user_input()
    print("Below are the Association rules :")
    print_association_rule(file_name, float(min_support), float(min_confidence))
    print()

For Database Number # 1 
Please provide minimum Support value (Only Value): 20
Please provide minimum Confidence value (Only Value): 50
Below are the Association rules :
Total number of items in association list is 6
['Knife'] -->> ['Blender'] : 100.0
['Blender'] -->> ['Knife'] : 100.0
['Watch'] -->> ['Tissue'] : 83.33333333333333
['Tissue'] -->> ['Watch'] : 100.0
['Watch'] -->> ['Umbrella'] : 66.66666666666667
['Umbrella'] -->> ['Watch'] : 100.0

For Database Number # 2 
Please provide minimum Support value (Only Value): 15
Please provide minimum Confidence value (Only Value): 70
Below are the Association rules :
Total number of items in association list is 10
['Knife'] -->> ['Blender'] : 100.0
['Blender'] -->> ['Knife'] : 100.0
['Pen'] -->> ['Pencil'] : 75.0
['Pencil'] -->> ['Pen'] : 75.0
['Headphone'] -->> ['Mobile'] : 75.0
['Laptop'] -->> ['Mobile'] : 75.0
['Pants'] -->> ['Sock'] : 100.0
['Sock'] -->> ['Pants'] : 100.0
['Laptop'] -->> ['Printer'] : 75.0
['Printer'] -->> ['Laptop'] 

In [17]:
import time
def compare_bruteforce_apriori(file_name, min_support, min_confidence):
    all_rows = input_database(file_name)
    unique_items = all_distinct_items(all_rows)
    num_unique_items = len(unique_items)
    data = string_to_int(all_rows, unique_items)
    
    start = time.time()
    freq_items = brute_force_method(data, num_unique_items, min_support, min_confidence)
    association_list = find_association_from_items(data, freq_items, unique_items, min_confidence)
    end = time.time()
    print("Required time for brute force method {}".format(end - start))
    
    start = time.time()
    freq_items = apriori_algorithm(data, num_unique_items, min_support, min_confidence)
    association_list = find_association_from_items(data, freq_items, unique_items, min_confidence)
    end = time.time()
    print("Required time for apriori Algorithm {}".format(end - start))
    
    print("---------------------------------------------------------------")

In [18]:
for index in range(1,6):
    print("For Database Number # {} :".format(index))
    file_name = 'database_{}.csv'.format(index)
    min_support, min_confidence = user_input()
    compare_bruteforce_apriori(file_name, float(min_support), float(min_confidence))

For Database Number # 1 :
Please provide minimum Support value (Only Value): 15
Please provide minimum Confidence value (Only Value): 60
19
171
969
3876
Required time for brute force method 0.05509328842163086
Required time for apriori Algorithm 0.0010380744934082031
---------------------------------------------------------------
For Database Number # 2 :
Please provide minimum Support value (Only Value): 20
Please provide minimum Confidence value (Only Value): 55
20
190
Required time for brute force method 0.005011081695556641
Required time for apriori Algorithm 0.0009703636169433594
---------------------------------------------------------------
For Database Number # 3 :
Please provide minimum Support value (Only Value): 18
Please provide minimum Confidence value (Only Value): 57
19
171
969
3876
Required time for brute force method 0.05783653259277344
Required time for apriori Algorithm 0.00199127197265625
---------------------------------------------------------------
For Database N